In [1]:
%%configure -f
{ "conf":{
        "spark.pyspark.python": "python3"
    ,"spark.pyspark.virtualenv.enabled": "true"
    ,"spark.pyspark.virtualenv.type":"native"
    ,"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    ,"spark.sql.files.ignoreCorruptFiles":"true"
    ,"spark.dynamicAllocation.executorIdleTimeout":"18000"
    ,"spark.driver.memory":"24g","spark.executor.memory":"24g"
    ,"spark.driver.cores":"5"
    ,"spark.driver.maxResultSize":"5g"
         }
}  

https://towardsdatascience.com/basics-of-apache-spark-configuration-settings-ca4faff40d45
https://luminousmen.com/post/spark-tips-partition-tuning
https://sparkbyexamples.com/pyspark/pyspark-repartition-vs-partitionby/

In [2]:
try:
    sc.install_pypi_package("aiohttp==3.8.1")
except: 
    print(f'aiohttp is installed')
try:
    import pandas as pd
except: 
    sc.install_pypi_package("pandas==1.1.5")
    import pandas as pd
try:
    import pyarrow
except: 
    sc.install_pypi_package("pyarrow==0.14.1")
    import pyarrow 
try:
    import s3fs
except: 
    sc.install_pypi_package("s3fs")
    import s3fs 
try:
    import fsspec
except: 
    sc.install_pypi_package("fsspec")
    import fsspec 
if False:
    try:
        import matplotlib
        import matplotlib.pyplot as plt
    except: 
        sc.install_pypi_package("matplotlib")
        import matplotlib
        import matplotlib.pyplot as plt
import pyspark.sql.functions as py_f
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1666113837887_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Running setup.py bdist_wheel for yarl: started
  Running setup.py bdist_wheel for yarl: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/33/f3/b3/19157d9ed9a5cb7e3c701ca6f99bd086ab162e4ef57729c008
  Running setup.py bdist_wheel for multidict: started
  Running setup.py bdist_wheel for multidict: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/e7/67/fd/4e3843a89ae0e233d79452f8f04e45814f350173a99f5addc9
Successfully built yarl multidict




In [3]:
print(spark.version,"\n\n")
configurations = spark.sparkContext.getConf().getAll()
for conf in configurations:
    print(conf)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.4.6-amzn-0 


('spark.driver.memory', '24g')
('spark.eventLog.enabled', 'true')
('spark.yarn.executor.memoryOverheadFactor', '0.1875')
('spark.executorEnv.PYTHONPATH', '{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.7-src.zip<CPS>{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.7-src.zip')
('spark.driver.extraLibraryPath', '/usr/lib/hadoop/lib/native:/usr/lib/hadoop-lzo/lib/native')
('spark.sql.parquet.output.committer.class', 'com.amazon.emr.committer.EmrOptimizedSparkSqlParquetOutputCommitter')
('spark.driver.cores', '5')
('spark.blacklist.decommissioning.timeout', '1h')
('spark.yarn.appMasterEnv.SPARK_PUBLIC_DNS', '$(hostname -f)')
('spark.sql.emr.internal.extensions', 'com.amazonaws.emr.spark.EmrSparkSessionExtensions')
('spark.yarn.secondary.jars', 'livy-api-0.7.0-incubating.jar,livy-rsc-0.7.0-incubating.jar,livy-thriftserver-session-0.7.0-incubating.jar,netty-all-4.1.17.Final.jar,commons-codec-1.9.jar,livy-core_2.11-0.7.0-incubating.jar,livy-repl_2.11-0.7.0-incubating.jar')
('spark.eventLo

In [6]:
one_file="s3://maystreetdata/feeds_norm/partition_scheme_experiments_7/mstnorm_parquet_0_5_0/mt_roundlot_bbo.parquet/"
d_df=spark.read.parquet(one_file)
d_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+----------------------------------+------------------------+------------------------+-----------------------+--------------------------------+--------------------------------+------------------------+-------+
|Feed_mt_roundlot_bbo|f_mt_roundlot_bbo|exchange_timestamp_mt_roundlot_bbo|best_bid_mt_roundlot_bbo|best_ask_mt_roundlot_bbo|bid_ask_mt_roundlot_bbo|timestamp_ts_utc_mt_roundlot_bbo|timestamp_ts_est_mt_roundlot_bbo|FeedType_mt_roundlot_bbo|Product|
+--------------------+-----------------+----------------------------------+------------------------+------------------------+-----------------------+--------------------------------+--------------------------------+------------------------+-------+
|                 CQS|       cqs_pillar|               1654070400014248704|                  52.690|                  52.950|               0.004935|             2022-06-01 08:00:00|             2022-06-01 04:00:00|            mt=bbo_quote|   URTY|
|   

In [9]:
f_array=['f=bats_edga',
'f=bats_edgx',
'f=byx',
'f=bzx',
'f=cts_pillar',
'f=iex_deep',
'f=memoir_depth',
'f=miax_pearl_equities_dom',
'f=total_view_bx',
'f=total_view_psx',
'f=total_view',
'f=utdf_binary',
'f=xdp_american_integrated',
'f=xdp_arca_integrated',
'f=xdp_chicago_integrated',
'f=xdp_national_integrated',
'f=xdp_nyse_integrated',]
root_dir = 's3://maystreetdata/feeds_norm/mstnorm_parquet_0_5_0/mt=trade/'
may_street_rootdir='s3://maystreetdata/feeds_norm/mstnorm_parquet_0_5_0'
may_street_feeds=[
'mt=add_order',
'mt=aggregated_price_update',
'mt=auction_summary',
'mt=bbo_quote',
'mt=cancel_order',
'mt=clear_orders',
'mt=feed_status',
'mt=index_update',
'mt=market_status',
'mt=missing_packets',
'mt=modify_order',
'mt=nbbo_quote',
'mt=order_imbalance',
'mt=price_level_update',
'mt=product_announcement',
'mt=product_statistics',
'mt=product_status',
'mt=retail_price_improvement',
'mt=trade_break',
'mt=trade_correction',
'mt=trade',]
f_array=['f=bats_edga/',
'f=bats_edgx/',
'f=byx/',
'f=bzx/',
'f=iex_deep/',
'f=memoir_depth/',
'f=miax_pearl_equities_dom/',
'f=total_view_bx/',
'f=total_view_psx/',
'f=total_view/',
'f=xdp_american_integrated/',
'f=xdp_arca_integrated/',
'f=xdp_chicago_integrated/',
'f=xdp_national_integrated/',
'f=xdp_nyse_integrated/',]
def get_data(root_dir,sub_dir_array):
    all_dfs={}
    data_df=None
    for one_feed in sub_dir_array:
        one_file = f'{root_dir}/{one_feed}/'
        try:
            one_df = spark.read.parquet(one_file)
            all_dfs[one_feed]=one_df
            if data_df is None:
                data_df=one_df
            else:
                data_df=data_df.union(one_df)
        except Exception as l_exc:
             all_dfs[one_feed]=f'exc:{l_exc}'
    return(all_dfs,data_df)
trade_df_array,trade_df=get_data('s3://maystreetdata/feeds_norm/mstnorm_parquet_0_5_0/mt=aggregated_price_update/',f_array)
#nbbo_df_array,nbbo_df=get_data('s3://maystreetdata/feeds_norm/mstnorm_parquet_0_5_0/mt=aggregated_price_update',[''])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
one_df = spark.read.parquet('s3://maystreetdata/feeds_norm/mstnorm_parquet_0_5_0/mt=aggregated_price_update/f=bats_edgx/')
one_df.show()

In [11]:
trade_df.groupby('Feed').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----------+
|         Feed|      count|
+-------------+-----------+
|    BatsPitch|14862165929|
|       MEMOIR| 4599698360|
|        XDPV2|14991765142|
|          IEX| 2116669098|
| MIAXEquities| 1237173174|
|TotalViewItch|12610378274|
+-------------+-----------+

In [18]:
res_dict={}
for k,v in trade_df_array.items():
    res_dict[k] = (v.count())
    
res_dict

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'f=bats_edga/': 1930735935, 'f=bats_edgx/': 5150372274, 'f=byx/': 1368287073, 'f=bzx/': 6412770647, 'f=iex_deep/': 2116669098, 'f=memoir_depth/': 4599698360, 'f=miax_pearl_equities_dom/': 1237173174, 'f=total_view_bx/': 1015498285, 'f=total_view_psx/': 1730156613, 'f=total_view/': 9864723376, 'f=xdp_american_integrated/': 1363483825, 'f=xdp_arca_integrated/': 7360890672, 'f=xdp_chicago_integrated/': 727316432, 'f=xdp_national_integrated/': 562030990, 'f=xdp_nyse_integrated/': 4978043223}

In [7]:
trade_df.rdd.getNumPartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

27802

In [8]:
l_fname=f"s3://maystreetdata/feeds_norm/partition_scheme_experiments_product_feed_dt/mstnorm_parquet_0_5_0/XDPV2_IEX_Total.parquet"
#trade_df.repartition(1).write\
trade_df.repartition(1000).write\
            .option("header",True) \
            .partitionBy("Product","Feed","dt") \
            .mode("overwrite") \
            .parquet(f"{l_fname}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o450.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:173)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:173)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:169)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:197)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at

In [7]:
                                                          #1636035828600286047
trade_df.where(f"Product=='{'SPY'}' and ExchangeTimestamp<{1661204000000431328}")\
.orderBy('ExchangeTimestamp','SequenceNumber').toPandas().to_csv('s3://maystreetdata/analysis/spy_sample.csv')#.agg(py_f.min(py_f.col('ExchangeTimestamp')),py_f.max(py_f.col('ExchangeTimestamp'))).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
nbbo_df.where(f"Product=='{'SPY'}' and LastExchangeTimestamp<{1661204000000431328}").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
spy_nbbo=nbbo_df.where(f"Product=='{'SPY'}'").select('Product','LastExchangeTimestamp','Feed', 'LastSequenceNumber', 'BidPrice_1','BidQuantity_1','AskPrice_1','AskQuantity_1')\
.withColumnRenamed('LastExchangeTimestamp','ExchangeTimestamp_nbbo').withColumnRenamed('Product','Product_nbbo').withColumnRenamed('LastSequenceNumber','SequenceNumber_nbbo')
spy_trade=trade_df.where(f"Product=='{'SPY'}'")

spy_nbbo_trade=spy_nbbo.join(spy_trade,(spy_nbbo.Product_nbbo==spy_trade.Product)  
                             & (spy_nbbo.Feed==spy_trade.Feed) 
                             & (spy_nbbo.ExchangeTimestamp_nbbo==spy_trade.ExchangeTimestamp) ,'outer').drop(spy_trade.Feed)
spy_nbbo_trade=spy_nbbo_trade.withColumn('exchange_ts',py_f.when(py_f.col(f'ExchangeTimestamp_nbbo').isNull(), py_f.col(f'ExchangeTimestamp')).otherwise(py_f.col(f'ExchangeTimestamp_nbbo')))
spy_nbbo_trade.orderBy('exchange_ts','SequenceNumber_nbbo','SequenceNumber', 'ProductSequenceNumber')\
.select('Product_nbbo','SequenceNumber','Feed','SequenceNumber_nbbo', 'exchange_ts','Side', 'BidQuantity_1', 'BidPrice_1','Price','AskPrice_1', 'AskQuantity_1','Quantity','SaleCondition','SaleCondition2','SaleCondition3','SaleCondition4')\
.where("SaleCondition3 !='ExtendedHoursTrade' ").toPandas().to_csv('s3://maystreetdata/analysis/spy_full.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
import pyarrow.parquet as pq


class ParquetInspector(object):
    """docstring for ParquetInspector"""

    def __init__(self, parquet_filepath):
        super(ParquetInspector, self).__init__()
        self.parquet_filepath = parquet_filepath

        self.metadata = pq.read_metadata(self.parquet_filepath)

    def footer_size(self):
        return self.metadata_serialized_size()

    def metadata_serialized_size(self):
        return self.metadata.serialized_size

    def row_count(self):
        return self.metadata.num_rows

parquet_path='s3://maystreetdata/feeds_norm/mstnorm_parquet_0_5_0/mt=aggregated_price_update/f=bats_edga/dt=2022-07-20/1.parquet'
inspector = ParquetInspector(parquet_path)
footer_size = inspector.footer_size()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Failed to open local file 's3://maystreetdata/feeds_norm/mstnorm_parquet_0_5_0/mt=aggregated_price_update/f=bats_edga/dt=2022-07-20/1.parquet', error: No such file or directory
Traceback (most recent call last):
  File "<stdin>", line 11, in __init__
  File "/tmp/1665164519839-0/lib/python3.7/site-packages/pyarrow/parquet.py", line 1423, in read_metadata
    return ParquetFile(where, memory_map=memory_map).metadata
  File "/tmp/1665164519839-0/lib/python3.7/site-packages/pyarrow/parquet.py", line 132, in __init__
    self.reader.open(source, use_memory_map=memory_map, metadata=metadata)
  File "pyarrow/_parquet.pyx", line 1002, in pyarrow._parquet.ParquetReader.open
  File "pyarrow/io.pxi", line 1323, in pyarrow.lib.get_reader
  File "pyarrow/io.pxi", line 1314, in pyarrow.lib._get_native_file
  File "pyarrow/io.pxi", line 818, in pyarrow.lib.OSFile.__cinit__
  File "pyarrow/io.pxi", line 828, in pyarrow.lib.OSFile._open_readable
  File "pyarrow/error.pxi", line 87, in pyarrow.lib.chec